<a href="https://colab.research.google.com/github/gonzalobarrazueta/TF-20181e098-20191a972-201920156/blob/main/MAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Crear diccionario de calles**
El archivo .csv contiene el `id` que le corresponde a cada calle. Guardo los datos en un diccionario llamado `streetsDicitonary`

In [1]:
import csv
import json

In [2]:
def addKey_Values(dictionary, key, value):
    dictionary[key] = value

In [3]:
def cargarDatos(dictionary):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        
        streetId = 0
        for row in reader:
            addKey_Values(dictionary, row[0].lower(), streetId)
            streetId += 1

In [4]:
def createStreetsDictionary():
    streets = {}
    # carga datos al diccionario 'streets'
    cargarDatos(streets)

    streets_pretty = json.dumps(streets, indent=2)
    print(streets_pretty)
    
    return streets

In [5]:
streetsDictionary = createStreetsDictionary()

{
  "prolongacion naranjal": 0,
  "entrada al templo huaca el paraiso": 1,
  "calle uranio": 2,
  "el sol de naranjal": 3,
  "avenida josefina": 4,
  "avenida naranjal": 5,
  "calle e": 6,
  "calle 28 de mayo": 7,
  "calle los petalos": 8,
  "avenida paramonga": 9,
  "santa cruz": 10,
  "santa rosa": 11,
  "calle las violetas": 12,
  "virgen del rosario": 13,
  "calle llata": 14,
  "san nicolas": 15,
  "los eucaliptos": 16,
  "los claveles": 17,
  "calle las palmas": 18,
  "las gardenias": 19,
  "circa patria": 20,
  "avenida central": 21,
  "avenida pacasmayo": 22,
  "los geranios 2": 23,
  "jiron las americas": 24,
  "miguel grau": 25,
  "avenida san jose": 26,
  "los portales de chavin": 27,
  "calle los claveles": 28,
  "calle los ficus": 29,
  "avenida canta callao": 30,
  "los nogales": 31,
  "el olivar": 32,
  "calle bolognesi": 33,
  "avenida los alisos": 34,
  "avenida santa rosa": 35,
  "san pablo": 36,
  "avenida antigua pacasmayo": 37,
  "avenida carlos izaguirre": 38,
  "c

###**Crear Lista de Adyacencia para las calles y sus intersecciones**
* -1 significa que la calle aún no está mapeada



In [6]:
def getLastValueInDictionary():
    lastStreet = list(streetsDictionary.keys())[-1]
    id = streetsDictionary[lastStreet]
    return id

In [7]:
def streetExists(streetName):
    if streetName in streetsDictionary:
        s = streetsDictionary[streetName]
    else:
        s = -1
        # agrega la calle al final del diccionario, si no existe
        #s = getLastValueInDictionary() + 1
        #streetsDictionary[streetName] = s
    return s

In [8]:
def getAmountOfIntersections(graph):
    totalIntersections = 0
    for intersections in graph:
        for i in range(len(intersections)):
            if (intersections[i] != -1):
                totalIntersections += 1
        #totalIntersections += len(intersections)
    
    print(totalIntersections)

def printAdList(graph):
    print(f"n = {len(intersectionsAdList)}")
    i = 0
    for intersection in intersectionsAdList:
        print(i, intersection)
        i += 1

In [9]:
# adjacencyList: arreglo de arreglos, donde la pos del primer elemento (1er arreglo)
# es la calle y el arreglo son todas las calles con las que se intersecta
def createAdjacencyList(adjacencyList):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)

        # Leo línea por línea
        for row in reader:  
            intersections = []
            for i in range(len(row)):
                if i != 0:
                    intersections.append(streetExists(row[i].lower()))

            adjacencyList.append(intersections)

    #for intersection in adjacencyList:
    #    print(intersection, end='\n')
    return adjacencyList

intersectionsAdList = createAdjacencyList([])

getAmountOfIntersections(intersectionsAdList)
printAdList(intersectionsAdList)

517
n = 177
0 [1, 2, 3, 4]
1 [0]
2 [0]
3 [0, 4, 15, 16, 9, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 29, 28, 5, 30, 25]
4 [9, 3, 5, 0, 84]
5 [4, 15, 6, 2, 9, 17, 19, 21, 7, 22, -1, 8, 24, 10, 11, 12, 13, -1, 14, -1, 30, 58]
6 [5, -1, -1]
7 [5, 110, -1, -1]
8 [5]
9 [4, 15, 3, 16, 32]
10 [5]
11 [5]
12 [5]
13 [5]
14 [5, -1]
15 [3, 9, 31, 32, 19, 51, 54, 34]
16 [3, 19, 32, 31, 9]
17 [3]
18 [3]
19 [3]
20 [3]
21 [3, 32]
22 [3, 32, 33, 34, 35, 36, 37, 38, 105]
23 [3, 39, 40]
24 [3, -1]
25 [3]
26 [3, 41, 42, 43, 44, 45, 46, 34]
27 [3]
28 [3]
29 [3, 47, 48]
30 [3, 58, 61, 34, 62, 38, 63, 22, 64, 88]
31 [15, 49, 16]
32 [15, 49, 16, 9]
33 [22, 50]
34 [58, 30, 65, 62, 26, 68, 22, 69, 70, 71]
35 [22, 69, 72, 73, 70, 37, 76]
36 [22, 72, 77, 37, 79, 80]
37 [35, 81, 36, 22]
38 [64, 22, 58, 30]
39 [23, 50]
40 [23]
41 [26, 52]
42 [26]
43 [26, 52]
44 [-1]
45 [26, -1]
46 [26, 65]
47 [29, 55, 56, 57, 58]
48 [29, 82]
49 [31, -1, 32]
50 [39, 107, 33]
51 [52, 53, -1, 15]
52 [51]
53 [51, 83, 54]
54 [53, -1, 15]


###**Lista de adyacencia de las intersecciones de intersecciones**

In [10]:
# sirve para crear las intersecciones, como keys, en un diccionario
# si las keys aún no han sido creadas, las agrega al diccionario
def createKeys(graph):

    intersectionsIDs = {}

    for i in range(len(graph)):
        for street in graph[i]:
            if (i, street) not in intersectionsIDs and street != -1:
                intersectionsIDs[(i, street)] = []

    return intersectionsIDs

In [11]:
def addIntersections(graph, i, j):  
    
    street = graph[i][j]

    if street != -1:
        # agrego 1 a la izquierda
        if j > 0 and graph[i][j - 1] != -1:
            tup = (i, graph[i][j - 1])
            map[(i, street)].append(tup)

        # agrego 1 a la derecha
        if j < len(graph[i]) - 1 and graph[i][j + 1] != -1:
            tup = (i, graph[i][j + 1])
            map[(i, street)].append(tup)

In [12]:
def addIntersectionsReversed(graph, i, j):  
    #    0  1   2 
    # 0 [3, 5, 17]

    # 3 [7, 14, 0, 25]

    street = graph[i][j] # street = 3

    # ahora voy al reverso de la lista

    if i != -1 and street != -1:
        #preguntar si i existe en 3
        if (street < len(graph)) and (i in graph[street]):
            tup = (street, i)
            map[(i, street)].append(tup)
            
            # encuentro el índice de 0
            k = graph[street].index(i) # k = 2

            if k > 0 and graph[street][k - 1] != -1:
                tup = (street, graph[street][k - 1])
                map[(i, street)].append(tup)
        
            if k < len(graph[street]) - 1 and graph[street][k + 1] != -1:
                tup = (street, graph[street][k + 1])
                map[(i, street)].append(tup)

In [13]:
def joinIntersections(graph):
    n = len(graph)
    
    for i in range(n):
        
        for j in range(len(graph[i])):

            addIntersections(graph, i, j)
            addIntersectionsReversed(graph, i, j)

In [14]:
# Diccionario que va a guardar la unión de todas las intersecciones
map = dict()
# crea las llaves en el diccionario "map"
map = createKeys(intersectionsAdList)

joinIntersections(intersectionsAdList)

amountNodes = 0 

for key, value in map.items():
    amountNodes += len(value)
    print(key, ":", value)

#print(f"Cantidad de nodos: {amountNodes}")

(0, 1) : [(0, 2), (1, 0)]
(0, 2) : [(0, 1), (0, 3), (2, 0)]
(0, 3) : [(0, 2), (0, 4), (3, 0), (3, 4)]
(0, 4) : [(0, 3), (4, 0), (4, 5), (4, 84)]
(1, 0) : [(0, 1), (0, 2)]
(2, 0) : [(0, 2), (0, 1), (0, 3)]
(3, 0) : [(3, 4), (0, 3), (0, 2), (0, 4)]
(3, 4) : [(3, 0), (3, 15), (4, 3), (4, 9), (4, 5)]
(3, 15) : [(3, 4), (3, 16), (15, 3), (15, 9)]
(3, 16) : [(3, 15), (3, 9), (16, 3), (16, 19)]
(3, 9) : [(3, 16), (3, 17), (9, 3), (9, 15), (9, 16)]
(3, 17) : [(3, 9), (3, 18), (17, 3)]
(3, 18) : [(3, 17), (3, 19), (18, 3)]
(3, 19) : [(3, 18), (3, 20), (19, 3)]
(3, 20) : [(3, 19), (3, 21), (20, 3)]
(3, 21) : [(3, 20), (3, 22), (21, 3), (21, 32)]
(3, 22) : [(3, 21), (3, 23), (22, 3), (22, 32)]
(3, 23) : [(3, 22), (3, 24), (23, 3), (23, 39)]
(3, 24) : [(3, 23), (3, 26), (24, 3)]
(3, 26) : [(3, 24), (3, 27), (26, 3), (26, 41)]
(3, 27) : [(3, 26), (3, 29), (27, 3)]
(3, 29) : [(3, 27), (3, 28), (29, 3), (29, 47)]
(3, 28) : [(3, 29), (3, 5), (28, 3)]
(3, 5) : [(3, 28), (3, 30)]
(3, 30) : [(3, 5), (3, 

###**Write to file**

In [15]:
# Write Streets Dictionary
def writeStreetsDictionary():
    with open("streetNames.txt", "w") as f:
        for key, value in streetsDictionary.items():
            f.write(key + "\n")

writeStreetsDictionary()

In [16]:
# Write Intersections Adjacency List
def writeStreetsAdList():
    with open("streetsAdList.txt", "w") as f:
        for streets in intersectionsAdList:
            for i in range(len(streets)):
                streets[i] = str(streets[i])
            
            f.write(', '.join(streets) + "\n")
        
writeStreetsAdList()

In [17]:
# Write street map
def writeStreetMap():
    with open("map.txt", "w") as f:
        
        for key, value in map.items():
            listWOCommas = (str(value).strip('[]')).replace(",", "")
            f.write(listWOCommas + '\n')
            
writeStreetMap()